# Colocating Sentinel-3 OLCI and Sentinal-2 Optical Data


## Read in Functions Needed

In [16]:
import ee
from datetime import datetime, timedelta
from shapely.geometry import Polygon, Point
import numpy as np
import subprocess
import requests
import pandas as pd

def get_matched_S2_image_ids(s3_image,boundary_geometry):
        s3_time = datetime.utcfromtimestamp(s3_image.get('system:time_start').getInfo() / 1000)
        
        # Define a time window for S2 search (±3 hours from S3 image time)
        start_time = s3_time - timedelta(hours=3)
        end_time = s3_time + timedelta(hours=3)

        # Query for S2 images within the time window and spatial extent of S3
        S2_collection = ee.ImageCollection('COPERNICUS/S2') \
        .filterDate(start_time, end_time) \
        .filterBounds(boundary_geometry)

        # Return the list of S2 image IDs
        return S2_collection.aggregate_array('system:index').getInfo()


def find_matched_satellite_images(S3_date_range, S3_spatial_extent, boundary_geometry):
    """
    Function to find matched Sentinel-2 image IDs for each Sentinel-3 image in the given date range and spatial extent.
    """

    # Define variables for Sentinel-3 query
    S3_product = 'COPERNICUS/S3/OLCI'

    # Query for Sentinel-3 data
    S3_collection = ee.ImageCollection(S3_product) \
        .filterDate(S3_date_range[0], S3_date_range[1]) \
        .filterBounds(boundary_geometry)

    # Convert S3_collection to a list of image IDs
    S3_image_ids = S3_collection.aggregate_array('system:index').getInfo()

    # List to store matched pairs
    matched_pairs = []

    # Loop through each S3 image ID and find matching S2 images
    for s3_image_id in S3_image_ids:
        s3_image = ee.Image(S3_collection.filter(ee.Filter.eq('system:index', s3_image_id)).first())
        matched_S2_image_ids = get_matched_S2_image_ids(s3_image,boundary_geometry)

        # Record each pair of matched S3 and S2 images
        for s2_image_id in matched_S2_image_ids:
            matched_pairs.append((s3_image_id, s2_image_id))

    return matched_pairs


def get_matched_S3_image_ids(s2_image,boundary_geometry):
        s2_time = datetime.utcfromtimestamp(s2_image.get('system:time_start').getInfo() / 1000)
        
        # Define a time window for S2 search (±3 hours from S3 image time)
        start_time = s2_time - timedelta(hours=0.5)
        end_time = s2_time + timedelta(hours=0.5)

        # Query for S2 images within the time window and spatial extent of S3
        S3_collection = ee.ImageCollection('COPERNICUS/S3/OLCI') \
        .filterDate(start_time, end_time) \
        .filterBounds(boundary_geometry)

        # Return the list of S2 image IDs
        return S3_collection.aggregate_array('system:index').getInfo()


def find_matched_satellite_images_s2(S2_date_range, S3_spatial_extent):
    """
    Function to find matched Sentinel-2 image IDs for each Sentinel-3 image in the given date range and spatial extent.
    """

    # Define variables for Sentinel-3 query
    S2_product = 'COPERNICUS/S2'
    
    # Create a rectangle from the S3_spatial_extent if boundary_geometry is not provided
    
    min_lon, min_lat, max_lon, max_lat = S3_spatial_extent
    boundary_geometry = ee.Geometry.Rectangle([min_lon, min_lat, max_lon, max_lat])
    
    # Query for Sentinel-2 data
    S2_collection = ee.ImageCollection(S2_product) \
        .filterDate(S2_date_range[0], S2_date_range[1]) \
        .filterBounds(boundary_geometry)

    # Convert S3_collection to a list of image IDs
    S2_image_ids = S2_collection.aggregate_array('system:index').getInfo()

    # List to store matched pairs
    matched_pairs = []

    # Loop through each S3 image ID and find matching S2 images
    for s2_image_id in S2_image_ids:
        s3_image = ee.Image(S2_collection.filter(ee.Filter.eq('system:index', s2_image_id)).first())
        matched_S3_image_ids = get_matched_S3_image_ids(s3_image,boundary_geometry)

        # Record each pair of matched S3 and S2 images
        for s3_image_id in matched_S3_image_ids:
            matched_pairs.append((s2_image_id, s3_image_id))

    return matched_pairs

def parse_gee_filename(gee_filename):
    parts = gee_filename.split('_')
    sensing_date = parts[0]
    tile_number = parts[2]
    return sensing_date, tile_number

# Function to get an access token
def get_access_token(username, password):
    url = 'https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token'
    data = {
        'grant_type': 'password',
        'username': username,
        'password': password,
        'client_id': 'cdse-public'
    }
    response = requests.post(url, data=data)
    response.raise_for_status()
    return response.json()['access_token']

# Function to query Sentinel-2 data from Copernicus Data Space
def query_sentinel2_data(sensing_start_date, tile_number, token):
    # Convert sensing_start_date to datetime object and format it for the query
    start_time = datetime.strptime(sensing_start_date, '%Y%m%dT%H%M%S')
    end_time = start_time + timedelta(hours=2)  # Adjust the time window as necessary
    start_time_str = start_time.strftime('%Y-%m-%dT%H:%M:%SZ')
    end_time_str = end_time.strftime('%Y-%m-%dT%H:%M:%SZ')

    # Construct the request URL with the contains function for tile number
    url = f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=contains(Name,'{tile_number}') and Collection/Name eq 'SENTINEL-2' and ContentDate/Start gt {start_time_str} and ContentDate/Start lt {end_time_str}"
    headers = {'Authorization': f'Bearer {token}'}

    # Make the API request
    response = requests.get(url, headers=headers)
    response.raise_for_status()

    return pd.DataFrame.from_dict(response.json()['value'])

def extract_correct_product_name(df, start_time, tile_number):
    # Adjusted regex pattern to match the filename format
    pattern = f'MSIL1C.*{start_time}.*_{tile_number}_'
    filtered_products = df[df['Name'].str.contains(pattern, regex=True)]
    
    
    # Return the first matching product name, or None if not found
    return filtered_products['Name'].iloc[0] if not filtered_products.empty else None, filtered_products['Id'].iloc[0] if not filtered_products.empty else None

def process_image_pair(s2_ee_image_id, token):
    sensing_start_date = s2_ee_image_id.split('_')[0]
    tile_number = s2_ee_image_id.split('_')[2]

    # Query the Copernicus Data Space
    df = query_sentinel2_data(sensing_start_date, tile_number, token)

    # Extract the correct MSIL1C product name
    return extract_correct_product_name(df, sensing_start_date, tile_number)

In [5]:
ee.Initialize()
S2_date_range = ['2018-06-01', '2018-06-02']
S2_spatial_extent = [-60, 75, -20, 83]  # Arctic region coordinates
matched_pairs = find_matched_satellite_images_s2(S2_date_range, S2_spatial_extent)
print("Matched S2 and S3 Image Pairs:", matched_pairs)

Matched S2 and S3 Image Pairs: [('20180601T151911_20180601T151910_T23XNE', 'S3B_20180601T151325_20180601T151625'), ('20180601T151911_20180601T151910_T24XVK', 'S3B_20180601T151325_20180601T151625'), ('20180601T151911_20180601T151910_T24XVL', 'S3B_20180601T151325_20180601T151625'), ('20180601T151911_20180601T151910_T24XWK', 'S3B_20180601T151325_20180601T151625'), ('20180601T151911_20180601T151910_T24XWL', 'S3B_20180601T151325_20180601T151625'), ('20180601T151911_20180601T151910_T24XWM', 'S3B_20180601T151325_20180601T151625'), ('20180601T151911_20180601T151910_T24XWN', 'S3B_20180601T151325_20180601T151625'), ('20180601T151911_20180601T151910_T25XDE', 'S3B_20180601T151325_20180601T151625'), ('20180601T151911_20180601T151910_T25XDF', 'S3B_20180601T151325_20180601T151625'), ('20180601T151911_20180601T151910_T25XDG', 'S3B_20180601T151325_20180601T151625'), ('20180601T151911_20180601T151910_T25XDH', 'S3B_20180601T151325_20180601T151625'), ('20180601T151911_20180601T151910_T25XED', 'S3B_2018060

Now we have filenames of Sentinel-2 and Sentinel-3 OLCI, we need to convert it into Copernicus file format as we did in last week.
In this example, we only take one from Sentinel-2 and one from Sentinel-3 OLCI to show the download process. Now we start with Sentinel-2. We start by getting its filename in Copernicus format.

In [17]:
username = 'ucfbwc0@ucl.ac.uk'
password = 'Psos28633682Sosp~~~~'

    # Get the access token

gee_filename = '20190228T215529_20190228T215527_T05WNU'
token = get_access_token(username, password)
process_image_pair(gee_filename, token)

('S2B_MSIL1C_20190228T215529_N0207_R029_T05WNU_20190228T233133.SAFE',
 'b4a388de-ae53-58f0-bebf-9d2367058c00')